In [1]:
import numpy as np
import pandas as pd

In [2]:
"""
Looking at population density and distribution of people

Dataset:
Base Zoning: http://opendata.columbus.gov/datasets/96f7642a62f84db997f9e1db4a776995_4
    - Can look at zoning to see which locations are more populated
    
BZA Zoning Variances: http://opendata.columbus.gov/datasets/19786dd084e644a4aea6b33f867dd631_1
    - More Zoning?
    
Recommended Land Use: http://opendata.columbus.gov/datasets/26f0606f94db4c07a63aef3cc8927c9b_21
    - Where we can build charging stations?
    
Population Density Map: https://apps.morpc.org/census2010/


Webscraping data:
    - https://github.com/ikhlaqsidhu/data-x/blob/master/03-tools-webscraping-crawling_api_afo/notebook-webscraping_v4.ipynb

API to convert Lat/Long to census block
    - https://geo.fcc.gov/api/census/#!/area/get_area
    
Conversion for GEOIDs
    - https://www.census.gov/geo/reference/codes/cou.html
    - https://www.census.gov/geo/reference/geoidentifiers.html (General info on how it's generated)

# TODO: find a way to associate GEOID with latitude/longitude, somehow associate with city/county?

""";

In [3]:
%ls

Data/                   PT_data.ipynb           README.md
NS_data.ipynb           Points_of_Interest.csv


In [4]:
census_data = pd.read_excel("Data/Columbus_Population.xlsx")

In [5]:
print(len(census_data))
census_data.head()

1489


,GEOID,2010 Total Population,2010 White Population,2010 African American Population,2010 Asian Population,2010 Other Races Population,2010 Hispanic Population,2010 Total Households,2010 Occupied Households,2010 Vacant Households,2000 Total Population,2000 White Population,2000 African American Population,2000 Asian Population,2000 Other Races Population,2000 Total Housing Units,2000 Occupied Housing Units,2000 Vacant Housing Units
0,390410101003,2258,1896,156,129,77,63,449,397,52,2579,2305,157,66,51,447,414,33
1,390410102002,1002,953,18,3,28,24,454,423,31,1085,1038,13,1,33,443,425,18
2,390410102003,2692,2446,127,30,89,82,951,932,19,1180,1137,23,9,11,524,439,85
3,390410102004,927,834,43,1,49,33,369,345,24,898,838,18,4,38,391,346,45
4,390410105201,229,217,3,2,7,3,107,98,9,218,211,0,0,7,102,98,4


In [6]:
geoid_conversion = pd.read_csv("Data/OhioGEOIDConversion.csv").iloc[:,:4]
geoid_conversion.head()

,State,StateCode,CountyCode,CountyName
0,OH,39,1,Adams County
1,OH,39,3,Allen County
2,OH,39,5,Ashland County
3,OH,39,7,Ashtabula County
4,OH,39,9,Athens County


In [7]:
def geoidToCountyCode(geoid):
    return int((geoid - 390000000000) / 10000000)

def geoidToCountyName(geoid):
    countyCode = geoidToCountyCode(geoid)
    return geoid_conversion[geoid_conversion["CountyCode"] == countyCode].CountyName.item()

geoids = census_data.iloc[:,0]
countyCodes = geoids.apply(geoidToCountyCode)
countyNames = geoids.apply(geoidToCountyName)
census_data.insert(1, "CountyNames", countyNames)
census_data.insert(2, "CountyCode", countyCodes)
census_data.head()

,GEOID,CountyNames,CountyCode,2010 Total Population,2010 White Population,2010 African American Population,2010 Asian Population,2010 Other Races Population,2010 Hispanic Population,2010 Total Households,2010 Occupied Households,2010 Vacant Households,2000 Total Population,2000 White Population,2000 African American Population,2000 Asian Population,2000 Other Races Population,2000 Total Housing Units,2000 Occupied Housing Units,2000 Vacant Housing Units
0,390410101003,Delaware County,41,2258,1896,156,129,77,63,449,397,52,2579,2305,157,66,51,447,414,33
1,390410102002,Delaware County,41,1002,953,18,3,28,24,454,423,31,1085,1038,13,1,33,443,425,18
2,390410102003,Delaware County,41,2692,2446,127,30,89,82,951,932,19,1180,1137,23,9,11,524,439,85
3,390410102004,Delaware County,41,927,834,43,1,49,33,369,345,24,898,838,18,4,38,391,346,45
4,390410105201,Delaware County,41,229,217,3,2,7,3,107,98,9,218,211,0,0,7,102,98,4


In [8]:
census_by_county = census_data.groupby(["CountyNames", "GEOID"]).sum()
census_by_county

CountyCode  2010 Total Population  \
CountyNames     GEOID                                             
Delaware County 390410101001          41                   1277   
                390410101002          41                   1721   
                390410101003          41                   2258   
                390410102001          41                    859   
                390410102002          41                   1002   
                390410102003          41                   2692   
                390410102004          41                    927   
                390410104201          41                    556   
                390410104202          41                   2370   
                390410104211          41                   2182   
                390410104221          41                   2374   
                390410104222          41                    772   
                390410104223          41                   2556   
                390410105201          41                    229   
                390410105202          41                   2742   
                390410105203          41                   2244   
                390410105301          41                   1601   
                390410105302          41                   1616   
                390410111011          41                   1190   
                390410111012          41                   1707   
                390410111021          41                   2485   
                390410111022          41                   2432   
                390410112001          41                   2395   
                390410112002          41                    881   
                390410114111          41                   1108   
                390410114112          41                   1793   
                390410114121          41                   2859   
                390410114122          41                   1658   
                390410114131          41                   1285   
                390410114132          41                   2842   
...                                  ...                    ...   
Union County    391590501001         159                   1454   
                391590501002         159                   1255   
                391590501003         159                   1178   
                391590502001         159                   1438   
                391590502002         159                    880   
                391590502003         159                   1610   
                391590502004         159                    782   
                391590503011         159                   1455   
                391590503012         159                   2430   
                391590503031         159                   1511   
                391590503032         159                   5137   
                391590503041         159                   1097   
                391590503042         159                   1249   
                391590504001         159                   2063   
                391590504002         159                   1708   
                391590504003         159                   2972   
                391590504004         159                   2597   
                391590505001         159                   1396   
                391590505002         159                   1856   
                391590505003         159                    545   
                391590505004         159                   1481   
                391590506011         159                    825   
                391590506012         159                   1116   
                391590506013         159                   2931   
                391590506021         159                   2227   
                391590506022         159                   1718   
                391590507001         159                   1075   
          

In [9]:
census_data.CountyCode.unique()

array([ 41,  45,  47,  49,  83,  89,  97, 101, 117, 129, 141, 159])